In [6]:
!mo --input_model age_net.caffemodel --input_proto age_deploy.prototxt --framework caffe
!mo --input_model gender_net.caffemodel --input_proto gender_deploy.prototxt --framework caffe


[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
Please expect that Model Optimizer conversion might be slow. You are currently using Python protobuf library implementation. 
Check that your protobuf package version is aligned with requirements_caffe.txt.


 For more information please refer to Model Conversion API FAQ, question #80. (https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_prepare_model_Model_Optimizer_FAQ.html?question=80#question-80)
[ INFO ] Generated IR will be compressed to FP16. If you get lower accu

In [2]:
import cv2 as cv
from openvino.runtime import Core
import numpy as np
import geocoder
from twilio.rest import Client
from geopy.geocoders import Nominatim
import winsound

TWILIO_ACCOUNT_SID = '*************'
TWILIO_AUTH_TOKEN = '***********************'
TWILIO_PHONE_NUMBER = '*************'
TARGET_PHONE_NUMBERS = ['*****************']

GENDER_LIST = ['Male', 'Female']
AGE_LIST = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(21,24)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']

single_woman_message_printed = False
lone_woman_between_men_message_printed = False
sos_triggered = False

core = Core()
age_model = core.read_model("age_net.xml")
gender_model = core.read_model("gender_net.xml")
age_compiled_model = core.compile_model(age_model, "CPU")
gender_compiled_model = core.compile_model(gender_model, "CPU")

face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

def send_sms(message):
    try:
        client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
        for target_phone_number in TARGET_PHONE_NUMBERS:
            client.messages.create(
                body=message,
                from_=TWILIO_PHONE_NUMBER,
                to=target_phone_number
            )
            print(f"SMS sent: {message}")
    except Exception as e:
        print(f"Failed to send SMS: {e}")

def get_location():
    g = geocoder.ip('me')
    return g.latlng

def get_detailed_location():
    g = geocoder.ip('me')
    if g.latlng:
        lat, lng = g.latlng
        geolocator = Nominatim(user_agent="myGeocoder")
        location = geolocator.reverse((lat, lng), language='en', timeout=10)
        if location:
            return location.address
    return "Location not available"

def preprocess_image(face, target_batch_size=10, is_age_model=False):
    blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    
    if is_age_model:
        return blob
    else:
        padded_blob = np.zeros((target_batch_size, *blob.shape[1:]), dtype=np.float32)
        padded_blob[0] = blob
        return padded_blob

def predict_age_gender_openvino(face_img):
    gender_blob = preprocess_image(face_img, target_batch_size=10, is_age_model=False)
    
    gender_infer_request = gender_compiled_model.create_infer_request()
    gender_infer_request.infer({0: gender_blob})
    gender_preds = gender_infer_request.get_output_tensor().data[0]
    gender = GENDER_LIST[np.argmax(gender_preds)]
    
    age_blob = preprocess_image(face_img, target_batch_size=1, is_age_model=True)
    
    age_infer_request = age_compiled_model.create_infer_request()
    age_infer_request.infer({0: age_blob})
    age_preds = age_infer_request.get_output_tensor().data[0]
    age = AGE_LIST[np.argmax(age_preds)]
    
    return gender, age

def detect_gesture(frame):
    global sos_triggered

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    blurred = cv.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv.threshold(blurred, 60, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    contours, _ = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    if contours:
        largest_contour = max(contours, key=cv.contourArea)
        area = cv.contourArea(largest_contour)

        
        if area > 6000:
            x, y, w, h = cv.boundingRect(largest_contour)

            if w > h * 1.5:
                if not sos_triggered:
                    sos_triggered = True
                    location = get_detailed_location()
                    send_sms(f"SOS Triggered!!! Location: {location}")
                    winsound.Beep(1000, 1000)
                    return "Thumb-Up (SOS Trigger)"

            elif h > w:
                if not sos_triggered:
                    sos_triggered = True
                    location = get_detailed_location()
                    send_sms(f"SOS Triggered!!! Location: {location}")
                    winsound.Beep(1500, 1000)
                    return "Fist Gesture (SOS Trigger)"
            else:
                pass
        else:
            pass
    else:
        pass
    
    return "Unknown"


def process_frame(frame):
    global single_woman_message_printed, lone_woman_between_men_message_printed

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    male_count = 0
    female_count = 0

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        gender, age = predict_age_gender_openvino(face_img)
        
        if gender == 'Male':
            male_count += 1
        elif gender == 'Female':
            female_count += 1
        
        label = f'{gender}, {age}'
        cv.putText(frame, label, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    gesture = detect_gesture(frame)
    
    count_label = f'Males: {male_count}, Females: {female_count}'
    cv.putText(frame, count_label, (10, frame.shape[0] - 40), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    if female_count == 1 and not single_woman_message_printed:
        print("Single woman detected")
        single_woman_message_printed = True  
    if male_count > 1 and female_count == 1 and not lone_woman_between_men_message_printed:
        print("Lone woman between men")
        lone_woman_between_men_message_printed = True  
    
    return frame

cap = cv.VideoCapture(0)

frame_width = 1280
frame_height = 720
cap.set(3, frame_width)
cap.set(4, frame_height)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = process_frame(frame)
    
    processed_frame_resized = cv.resize(processed_frame, (frame_width, frame_height))
    
    cv.imshow('Adyant Project N', processed_frame_resized)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


SMS sent: SOS Triggered!!! Location: Jodhpur, Jodhpur Tehsil, Jodhpur, Rajasthan, 112001, India
